In [5]:
import os
import json
from dotenv import load_dotenv
from scraper import fetch_website_contents, fetch_website_links
from openai import OpenAI
from IPython.display import display, Markdown, update_display



In [6]:
load_dotenv(override=True)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GEMINI_BASE = "https://generativelanguage.googleapis.com/v1beta/openai/"
GROQ_BASE = "https://api.groq.com/openai/v1"

groq_client = OpenAI(api_key=GROQ_API_KEY, base_url=GROQ_BASE)
google_client = OpenAI(api_key=GOOGLE_API_KEY, base_url=GEMINI_BASE)

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages. Include All the links that were passed into relevant type.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
def select_relevant_links(model,url):
    print(f"Selecting relevant links for {url} by calling {model}")
    if model == "openai/gpt-oss-20b":
        openai = groq_client
    elif model == "gemini-2.5-flash":
        openai = google_client

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
def fetch_page_and_all_relevant_links(model,url):
    contents = fetch_website_contents(url)
    print(contents)
    relevant_links = select_relevant_links(model,url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [11]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [12]:
def get_brochure_user_prompt(company_name, model,url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(model,url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [13]:
def create_brochure(company_name, Model, url):
    if Model == "Groq":
        openai = groq_client
        model = "openai/gpt-oss-20b"
    elif Model == "Gemini":
        openai = google_client
        model = "gemini-2.5-flash"
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, model, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [14]:
create_brochure("HuggingFace", "Groq","https://huggingface.co")

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
about 13 hours ago
•
2.44k
•
685
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
8 days ago
•
44.1k
•
681
Lightricks/LTX-2
Updated
1 day ago
•
1.19M
•
1.04k
openbmb/AgentCPM-Explore
Updated
1 day ago
•
315
•
287
Kijai/LTXV2_comfy
Updated
1 day ago
•
41.1k
•
279
Browse 2M+ models
Spaces
Running
on
Zero
Featured
787
Qwen Image Multiple Angles 3D Camera
🎥
787
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.47k
Z Image Turbo
🖼
1.47k
Generate stunning images from text descriptions in seconds
Running
Featured
4.2k
Wan2.2 Animate
👁
4.2k
Wan2.2 Animate
Running
on
Zero
MCP
Featured
391
Qwen-

# Hugging Face – The AI Community Building the Future  

---

## Who We Are  
Hugging Face is the world’s first open‑source collaboration hub for machine‑learning.  
- **Platform** where researchers, developers, and companies share, discover, and improve models, datasets, and end‑to‑end applications.  
- Home to **over 2 million models, 1 million+ apps (“Spaces”), and 500 k+ datasets** spanning text, image, video, audio, and 3D.  
- Built on the **HF Open‑Source stack** – no‑code, zero‑compute demos, and the ability to spin up paid compute for production workloads.

---

## What We Offer  

| Feature | Highlights |
|---------|------------|
| **Models** | Browse, fine‑tune, and deploy state‑of‑the‑art models (e.g., GLM‑Image, Qwen‑Image‑Edit, LTX‑2). |
| **Datasets** | Curated collections from academia, industry, and the community; fast, easy loading in Hugging Face 🤗. |
| **Spaces** | Instant demos: 3D image editing, text‑to‑image, video generation, and more—all running on free “Zero” compute. |
| **Enterprise** | Managed hosting, security, governance, and integration for large‑scale ML deployments. |
| **Docs & Community** | Rich tutorials, forums, Discord, and Hugging Face Hub for collaborative learning. |

---

## Culture & Community  

- **Open‑source first** – every model, dataset, or Space can be forked, improved, and shared back to the community.  
- **Collaboration at scale** – contributors from research labs, startups, and Fortune 500 firms build together.  
- **Inclusivity & Accessibility** – multilingual documentation, low‑latency demos, and support for under‑represented regions.  
- **Fast‑forward Innovation** – rapid iteration cycles, community‑curated benchmarks, and a commitment to responsible AI.

---

## Who Uses Hugging Face  

| Sector | Example Use Cases |
|--------|-------------------|
| **Research** | Benchmarking NLP, CV, and multimodal models; reproducible experiments. |
| **Start‑ups** | Rapid prototyping of AI products; embedding models into SaaS. |
| **Enterprise** | Customer‑support chatbots, content moderation, medical imaging diagnostics. |
| **Education** | Teaching ML concepts with hands‑on code; building student portfolios. |

---

## Careers at Hugging Face  

Hugging Face is growing fast – we’re hiring engineers, researchers, product managers, and community builders.  
- **Open Positions** – Explore current opportunities on our careers page and join a team that thrives on experimentation and impact.  
- **What We Value** – Curiosity, ownership, and a passion for building tools that make AI accessible to all.  
- **Benefits** – Remote‑first flexibility, equity, generous leave, and a culture that celebrates learning and collaboration.

---

## Ready to Build the Future Together?  

- **Sign Up** – Join the Hugging Face Hub, upload a model, dataset, or Space.  
- **Explore** – Browse 2 million+ models or 1 million+ apps to spark ideas.  
- **Collaborate** – Contribute to the community, fork projects, and share your work.

**Hugging Face** – Where machine‑learning innovators unite to create, share, and accelerate AI for everyone.